In [ ]:
import os
from dotenv import load_dotenv
import asyncio
import hashlib
from typing import Dict, Any, List
from redis.asyncio import Redis
from redisvl.extensions.cache.llm import SemanticCache
from redisvl.utils.vectorize import HFTextVectorizer
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer
import msgpack
import numpy as np

In [ ]:
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [ ]:
class SemanticLLMCache:
    def __init__(self, redis_url: str, model_name: str = "all-MiniLM-L6-v2"):
        self.redis = Redis.from_url(redis_url)
        self.embedder = HFTextVectorizer(
            model=model_name, 
            device="cpu"  # GPU для prod
        )
        
        # ✅ RedisVL SemanticCache (state-of-the-art 2025)
        self.semantic_cache = SemanticCache(
            name="llm_semantic_cache",
            redis_url=redis_url,
            distance_threshold=0.85,  # 85% семантическая схожесть = HIT
            vectorizer=self.embedder,
            redis_embedding_namespace="semantic_embeddings"
        )
        
        self.llm = ChatOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=OPENROUTER_API_KEY,
            model="grok-beta",
            temperature=0.1
        )
    
    async def get_or_generate(
        self, 
        messages: List[Dict[str, str]], 
        **kwargs
    ) -> Dict[str, Any]:
        """Семантическое кэширование + fallback на exact match"""
        
        # 1. Семантический поиск
        prompt_text = self._normalize_prompt(messages)
        cached = await self.semantic_cache.acheck(prompt_text)
        cached = cached[0]
        
        if cached:
            print(cached)
            print(f"✅ SEMANTIC HIT: {cached['response'][:50]}...")
            return cached
        
        # 2. Exact match fallback (SHA256)
        exact_key = self._make_exact_key(messages, **kwargs)
        exact_cached = await self._exact_cache_get(exact_key)
        if exact_cached:
            print(f"✅ EXACT HIT: {exact_key[:16]}...")
            return exact_cached
        
        # 3. LLM вызов
        print("🔄 MISS → LLM...")
        response = await self.llm.ainvoke(messages, **kwargs)
        
        # 4. Сохраняем в оба кэша
        await self.semantic_cache.astore(prompt_text, response.content)
        await self._exact_cache_set(exact_key, response)
        
        return {"content": response.content, "usage": response.response_metadata}
    
    def _normalize_prompt(self, messages: List[Dict]) -> str:
        """Канонический текст для эмбеддинга"""
        return " | ".join([f"{m['role']}: {m['content']}" for m in messages])
    
    def _make_exact_key(self, messages: List[Dict], **kwargs) -> str:
        payload = msgpack.packb({"messages": messages, **kwargs})
        return f"exact:{hashlib.sha256(payload).hexdigest()}"
    
    async def _exact_cache_get(self, key: str) -> Dict[str, Any] | None:
        data = await self.redis.get(key)
        return msgpack.unpackb(data, raw=False) if data else None
    
    async def _exact_cache_set(self, key: str, response: Dict, ttl: int = 7200):
        await self.redis.set(key, msgpack.packb(response), ex=ttl)

# 🧪 Тест семантического кэша
async def test_semantic_cache():
    cache = SemanticLLMCache("redis://localhost:6380")
    
    # Похожие запросы (разные формулировки)
    queries = [
        "Как работает Python async/await?",
        "Объясни асинхронность в Python",
        "Что такое asyncio в Python?"
    ]
    
    for i, q in enumerate(queries):
        messages = [{"role": "user", "content": q}]
        result = await cache.get_or_generate(messages)
        print(result.keys())
        print(f"Q{i+1}: {result['response'][:100]}...")
        await asyncio.sleep(0.1)  # Rate limit

await test_semantic_cache()


20:45:13 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: all-MiniLM-L6-v2
20:45:16 redisvl.index.index INFO   Index already exists, not overwriting.
20:45:16 redisvl.utils.vectorize.base WARNING   This vectorizer has no async embed method. Falling back to sync.
{'entry_id': 'c60b06d91e39e3f8df14a5e99049585be889279752ce0ed3a50186079e2b2737', 'prompt': 'user: Как работает Python async/await?', 'response': '`async` и `await` в Python используются для работы с асинхронным программированием, что позволяет выполнять операции ввода-вывода (I/O) без блокировки основного потока выполнения. Это особенно полезно для задач, которые могут занять много времени, таких как сетевые запросы или операции с файлами.\n\n### Основные концепции\n\n1. **Асинхронные функции**:\n   Функции, определенные с помощью ключевого слова `async`, являются асинхронными. Они возвращают объект типа `coroutine`, который можно запускать с помощью `await`.\n\n   ```python\n   async def my

IndexError: list index out of range

In [26]:
REDIS_URL = "redis://localhost:6380"

In [ ]:
from redis.asyncio import Redis, ConnectionPool


class LLMCache:
    def __init__(self, redis_url=REDIS_URL):
        self.redis_url = redis_url
        self.pool = ConnectionPool(self.redis_url)
        self.redis = Redis(connection_pool=self.pool)
        self.default_ttl = 3600

    def make_cache_key(self, messages):
        payload = {
            "messages": messages
        }
        key_hash = hashlib.sha256(payload).hexdigest()
        return f"llm:{key_hash}"
    
    async def get(self, key):
        data = await self.redis.get(key)
        if data:
            return data
        return None
    
    async def set(self, key, response, ttl = None):
        ttl = ttl or self.default_ttl
        data = response
        await self.redis.set(key, data, ex=ttl)

    async def get_or_call(self, key, ttl = None):
        pass